# Preprocesamiento IV

## 7. Formato YOLOv4

### 6.1. convert_coco_to_yolov4
Converts a COCO JSON annotation file into YOLOv4 format.

#### i. Convertir formato

In [1]:
import os
import json

In [2]:
def convert_coco_to_yolov4(coco_json_path, output_path):
    """
    Converts a COCO JSON annotation file into YOLOv4 format.

    Parameters:
        coco_json_path (str): Path to the COCO JSON file.
        output_path (str): Directory where YOLOv4 label files will be saved.
    """
    with open(coco_json_path, 'r', encoding='utf-8') as f:
        coco_data = json.load(f)

    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Create dictionary for image dimensions
    image_info = {img['id']: img for img in coco_data['images']}

    # Create dictionary for category mapping
    category_mapping = {cat['id']: idx for idx, cat in enumerate(coco_data['categories'])}

    # Group annotations by image
    annotations_by_image = {}
    for ann in coco_data['annotations']:
        img_id = ann['image_id']
        annotations_by_image.setdefault(img_id, []).append(ann)

    for img_id, annotations in annotations_by_image.items():
        img_data = image_info[img_id]
        file_name = os.path.splitext(img_data['file_name'])[0] + '.txt'
        txt_path = os.path.join(output_path, file_name)

        with open(txt_path, 'w', encoding='utf-8') as f:
            for ann in annotations:
                cat_id = ann['category_id']
                bbox = ann['bbox']  # [x_min, y_min, width, height]
                x_center = (bbox[0] + bbox[2] / 2) / img_data['width']
                y_center = (bbox[1] + bbox[3] / 2) / img_data['height']
                width = bbox[2] / img_data['width']
                height = bbox[3] / img_data['height']
                class_id = category_mapping[cat_id]
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


In [ ]:
convert_coco_to_yolov4('datasets/unified_dataset/redim/train.json', 'datasets/unified_dataset/redim_yolo/train')

In [ ]:
convert_coco_to_yolov4('datasets/unified_dataset/redim/train_augmented.json', 'datasets/unified_dataset/redim_yolo/train_augmented')

In [ ]:
convert_coco_to_yolov4('datasets/unified_dataset/redim/test.json', 'datasets/unified_dataset/redim_yolo/test')

In [ ]:
convert_coco_to_yolov4('datasets/unified_dataset/redim/valid.json', 'datasets/unified_dataset/redim_yolo/valid')

#### ii. Verificar cantidad de archivos

In [19]:
import os

def count_txt_files(input_path):
    """
    Counts the number of .txt files in the specified directory.

    Parameters:
        input_path (str): Path to the directory.

    Returns:
        int: Number of .txt files found.
    """
    count = sum(
        1 for file in os.listdir(input_path)
        if os.path.isfile(os.path.join(input_path, file)) and file.lower().endswith('.txt')
    )
    print(f"Total .txt files in '{input_path}': {count}")
    return count


In [ ]:
count_txt_files("datasets/unified_dataset/redim_yolo/train")

Total .txt files in 'datasets/unified_dataset/redim-yolo/train': 665


665

In [31]:
count_txt_files("datasets/unified_dataset/redim_yolo/train_augmented/labels")

Total .txt files in 'datasets/unified_dataset/redim_yolo/train_augmented/labels': 3325


3325

In [32]:
count_txt_files("datasets/unified_dataset/redim_yolo/valid/labels")

Total .txt files in 'datasets/unified_dataset/redim_yolo/valid/labels': 83


83

In [33]:
count_txt_files("datasets/unified_dataset/redim_yolo/test/labels")

Total .txt files in 'datasets/unified_dataset/redim_yolo/test/labels': 84


84

### 6.2. copy_coco_images
Copies only the images referenced in the COCO JSON to the output directory.

In [1]:
import os
import json
import shutil

In [2]:
def copy_coco_images(coco_json_path, input_image_path, output_image_path):
    """
    Copies only the images referenced in the COCO JSON to the output directory.

    Parameters:
        coco_json_path (str): Path to the COCO JSON file.
        input_image_path (str): Directory where original images are located.
        output_image_path (str): Directory where referenced images will be copied.
    """
    with open(coco_json_path, 'r', encoding='utf-8') as f:
        coco_data = json.load(f)

    os.makedirs(output_image_path, exist_ok=True)

    image_filenames = {img['file_name'] for img in coco_data['images']}

    for file_name in image_filenames:
        src_path = os.path.join(input_image_path, file_name)
        dst_path = os.path.join(output_image_path, file_name)

        # Create destination subdirectories if needed
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)

        if os.path.exists(src_path):
            shutil.copy2(src_path, dst_path)
        else:
            print(f"[WARNING] File not found: {src_path}")


In [3]:
copy_coco_images('datasets/unified_dataset/subsets/test.json', 'datasets/unified_dataset/images', 'datasets/unified_dataset/test')

### 6.3. create_yolo_image_list_path
Generates a .txt file listing full image paths from COCO JSON using a base path.

In [25]:
import os
import json

In [26]:
def create_yolo_image_list_path(base_path, coco_json_path, output_txt_path):
    """
    Generates a .txt file listing full image paths from COCO JSON using a base path.

    Parameters:
        base_path (str): Base path to prepend to each image file name.
        coco_json_path (str): Path to the COCO JSON file.
        output_txt_path (str): Path to the output .txt file (including the file name).
    """
    with open(coco_json_path, 'r', encoding='utf-8') as f:
        coco_data = json.load(f)

    image_filenames = [img['file_name'] for img in coco_data['images']]

    with open(output_txt_path, 'w', encoding='utf-8') as out_file:
        for file_name in image_filenames:
            image_path = os.path.join(base_path, file_name)
            out_file.write(f"{image_path}\n")


In [27]:
create_yolo_image_list_path(
    base_path='data/plate/',
    coco_json_path='datasets/unified_dataset/redim/valid.json',
    output_txt_path='datasets/unified_dataset/redim_yolo/valid.txt'
)


In [29]:
create_yolo_image_list_path(
    base_path='data/plate/',
    coco_json_path='datasets/unified_dataset/redim/train.json',
    output_txt_path='datasets/unified_dataset/redim_yolo/train.txt'
)


### 6.4. display_image_dimensions_with_frequency
Loads a COCO JSON and displays total image count, frequency and proportional frequency of each (width, height) dimension.

In [1]:
import json
from collections import Counter

In [2]:
def display_image_dimensions_with_stats(coco_json_path):
    """
    Loads a COCO JSON and displays total image count,
    frequency and proportional frequency of each (width, height) dimension.

    Parameters:
        coco_json_path (str): Path to the COCO-format JSON file.
    """
    with open(coco_json_path, 'r', encoding='utf-8') as f:
        coco_data = json.load(f)

    dimensions = [(img['width'], img['height']) for img in coco_data.get('images', [])]
    total_images = len(dimensions)
    dimension_counts = Counter(dimensions)

    print(f"Total number of images: {total_images}\n")
    print("Image dimensions with frequency and proportional frequency:")
    for dim, count in sorted(dimension_counts.items()):
        proportion = (count / total_images) * 100
        print(f"{dim}: {count} images ({proportion:.2f}%)")


In [3]:
display_image_dimensions_with_stats('datasets/unified_dataset/_annotations.coco.json')

Total number of images: 832

Image dimensions with frequency and proportional frequency:
(640, 640): 237 images (28.49%)
(1536, 2048): 2 images (0.24%)
(2048, 1536): 1 images (0.12%)
(3000, 4000): 472 images (56.73%)
(4000, 3000): 113 images (13.58%)
(6000, 8000): 7 images (0.84%)


### 6.5. resize_yolo_images_and_labels
Reize images and do modifications to the labels

In [34]:
import os
import cv2

def resize_yolo_images_and_labels(input_path, output_path, target_size=(608, 608)):
    os.makedirs(output_path, exist_ok=True)
    target_w, target_h = target_size

    for file in os.listdir(input_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_path, file)
            label_path = os.path.join(input_path, os.path.splitext(file)[0] + '.txt')

            image = cv2.imread(image_path)
            if image is None:
                continue

            h_orig, w_orig = image.shape[:2]

            resized_image = cv2.resize(image, (target_w, target_h))
            output_image_path = os.path.join(output_path, file)
            cv2.imwrite(output_image_path, resized_image)

            if os.path.exists(label_path):
                output_label_path = os.path.join(output_path, os.path.splitext(file)[0] + '.txt')
                with open(label_path, 'r') as f_in, open(output_label_path, 'w') as f_out:
                    for line in f_in:
                        parts = line.strip().split()
                        if len(parts) != 5:
                            continue
                        cls, x_center, y_center, width, height = map(float, parts)

                        # Convert from relative to absolute
                        x_abs = x_center * w_orig
                        y_abs = y_center * h_orig
                        w_abs = width * w_orig
                        h_abs = height * h_orig

                        # Resize absolute values according to new size
                        x_resized = x_abs * (target_w / w_orig)
                        y_resized = y_abs * (target_h / h_orig)
                        w_resized = w_abs * (target_w / w_orig)
                        h_resized = h_abs * (target_h / h_orig)

                        # Convert back to relative
                        x_new = x_resized / target_w
                        y_new = y_resized / target_h
                        w_new = w_resized / target_w
                        h_new = h_resized / target_h

                        f_out.write(f"{int(cls)} {x_new:.6f} {y_new:.6f} {w_new:.6f} {h_new:.6f}\n")


In [35]:
resize_yolo_images_and_labels('datasets/unified_dataset/yolo_upload/plate_all_dimensions', 'datasets/unified_dataset/yolo_upload/plate', target_size=(640, 640))

### 6.6 save_yolo_bboxes_to_images
Save images with bounding boxes drawn from YOLO annotations.

In [8]:
def save_yolo_bboxes_to_images(input_dir, output_dir='output', max_images=None):
    """
    Save images with bounding boxes drawn from YOLO annotations.

    Parameters:
        input_dir (str): Directory containing the images and YOLO .txt annotation files.
        output_dir (str): Directory to save output images with drawn bounding boxes.
        max_images (int, optional): If provided, randomly selects up to this number of images (without repetition).
    """
    from PIL import Image, ImageDraw
    import os
    import random

    os.makedirs(output_dir, exist_ok=True)

    # List all image files
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if max_images is not None and max_images < len(image_files):
        image_files = random.sample(image_files, max_images)

    for image_file in image_files:
        image_path = os.path.join(input_dir, image_file)
        label_path = os.path.join(input_dir, os.path.splitext(image_file)[0] + '.txt')
        output_path = os.path.join(output_dir, image_file)

        if not os.path.exists(label_path):
            print(f"[WARNING] Annotation file not found for image: {image_file}")
            continue

        image = Image.open(image_path).convert("RGB")
        draw = ImageDraw.Draw(image)
        w_img, h_img = image.size

        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                _, x_center, y_center, width, height = map(float, parts)

                # Convert YOLO normalized format to absolute pixel coordinates
                x_center *= w_img
                y_center *= h_img
                width *= w_img
                height *= h_img

                x0 = x_center - width / 2
                y0 = y_center - height / 2
                x1 = x_center + width / 2
                y1 = y_center + height / 2

                draw.rectangle([x0, y0, x1, y1], outline='red', width=3)

        image.save(output_path)


In [38]:
save_yolo_bboxes_to_images('datasets/unified_dataset/yolo_upload/plate', output_dir='datasets/unified_dataset/yolo_upload/plate_bbox')

### 6.7. show_yolo_bboxes
Displays a single image with its YOLO-format bounding boxes.

In [7]:
def show_yolo_bboxes(image_path, annotations_dir):
    """
    Displays a single image with its YOLO-format bounding boxes.

    Parameters:
        image_path (str): Full path to the image file.
        annotations_dir (str): Directory containing YOLO .txt annotations.
    """
    import os
    from PIL import Image, ImageDraw

    if not os.path.exists(image_path):
        print(f"[ERROR] Image not found: {image_path}")
        return

    filename = os.path.splitext(os.path.basename(image_path))[0]
    annotation_path = os.path.join(annotations_dir, filename + '.txt')

    if not os.path.exists(annotation_path):
        print(f"[WARNING] Annotation file not found for image: {image_path}")
        return

    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    w_img, h_img = image.size

    with open(annotation_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            _, x_center, y_center, width, height = map(float, parts)

            # Convert to absolute coordinates
            x_center *= w_img
            y_center *= h_img
            width *= w_img
            height *= h_img

            x0 = x_center - width / 2
            y0 = y_center - height / 2
            x1 = x_center + width / 2
            y1 = y_center + height / 2

            draw.rectangle([x0, y0, x1, y1], outline='red', width=3)

    image.show()


In [30]:
show_yolo_bboxes('datasets/unified_dataset/yolo_upload/plate_all_dimensions/20231009_193031.jpg', 'datasets/unified_dataset/yolo_upload/plate_all_dimensions/')

In [36]:
show_yolo_bboxes('datasets/unified_dataset/yolo_upload/plate/20231009_193031.jpg', 'datasets/unified_dataset/yolo_upload/plate/')

In [9]:
show_yolo_bboxes('datasets/unified_dataset/redim/train/_129602646_img_4997.jpg', 'datasets/unified_dataset/redim_yolo/train/')

### 6.8. get_unique_image_extensions
Reads a COCO-format JSON file and prints the unique image file extensions.

In [1]:
import os
import json

def get_unique_image_extensions(coco_json_path):
    """
    Reads a COCO-format JSON file and prints the unique image file extensions.

    Parameters:
        coco_json_path (str): Path to the COCO JSON annotation file.
    """
    with open(coco_json_path, 'r', encoding='utf-8') as f:
        coco_data = json.load(f)

    extensions = set()
    for image in coco_data.get("images", []):
        file_name = image.get("file_name", "")
        _, ext = os.path.splitext(file_name)
        if ext:
            extensions.add(ext.lower())

    print("Unique image file extensions found in the dataset:")
    for ext in sorted(extensions):
        print(ext)


In [3]:
get_unique_image_extensions('datasets/unified_dataset/_annotations.coco.json')

Unique image file extensions found in the dataset:
.jpg


## 8. Formato Faster RCNN

In [1]:
import os
import json

In [ ]:
def increment_category_ids_coco(input_json_path, output_json_path):
    # Crear carpeta de salida si no existe
    output_dir = os.path.dirname(output_json_path)
    os.makedirs(output_dir, exist_ok=True)

    # Cargar el COCO JSON
    with open(input_json_path, 'r') as f:
        coco_data = json.load(f)

    # Incrementar category_id en anotaciones
    for annotation in coco_data.get('annotations', []):
        annotation['category_id'] += 1

    # Incrementar id en categories (opcional, solo si quieres también ajustar categorías)
    for category in coco_data.get('categories', []):
        category['id'] += 1

    # Guardar el nuevo COCO JSON
    with open(output_json_path, 'w') as f:
        json.dump(coco_data, f)

    print(f"[INFO] Nuevo COCO JSON guardado en: {output_json_path}")




In [8]:
# Ejemplo de uso:
increment_category_ids_coco("datasets/unified_dataset/redim/train.json", "datasets/unified_dataset/redim_faster/train.json")

[INFO] Nuevo COCO JSON guardado en: datasets/unified_dataset/redim_faster/train.json


In [9]:
# Ejemplo de uso:
increment_category_ids_coco("datasets/unified_dataset/redim/valid.json", "datasets/unified_dataset/redim_faster/valid.json")

[INFO] Nuevo COCO JSON guardado en: datasets/unified_dataset/redim_faster/valid.json


In [10]:
# Ejemplo de uso:
increment_category_ids_coco("datasets/unified_dataset/redim/test.json", "datasets/unified_dataset/redim_faster/test.json")

[INFO] Nuevo COCO JSON guardado en: datasets/unified_dataset/redim_faster/test.json


In [11]:
# Ejemplo de uso:
increment_category_ids_coco("datasets/unified_dataset/redim/train_augmented.json", "datasets/unified_dataset/redim_faster/train_augmented.json")

[INFO] Nuevo COCO JSON guardado en: datasets/unified_dataset/redim_faster/train_augmented.json


## 9. Imagenes de Prueba

In [1]:
import os
import pandas as pd
import shutil

In [4]:
def copy_images_from_csv(csv_path, input_dir, output_dir, image_column="Nombre", extensions=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")):
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_csv(csv_path)
    names = df[image_column].dropna().unique()

    for name in names:
        found = False
        for ext in extensions:
            candidate = os.path.join(input_dir, name + ext)
            if os.path.isfile(candidate):
                shutil.copy(candidate, os.path.join(output_dir, name + ext))
                found = True
                break
        if not found:
            print(f"Image not found for: {name}")

In [5]:
csv_path = "files/imagenesPrueba-Hoja1.csv"
input_dir = "datasets/unified_dataset/images"
output_dir = "datasets/unified_dataset/O4_test_images"
copy_images_from_csv(csv_path, input_dir, output_dir)